# Exercise 1: Solr query on User Defined Type (UDT)

### Run the following CQL commands to add a UDT column and insert some data:

```
CREATE TYPE amazon.category_rank (
    category text,
    rank int
);

ALTER TABLE amazon.metadata ADD ranks set<frozen<category_rank>>;

UPDATE amazon.metadata SET ranks = ranks + { { category:'Movies', rank:1 } } WHERE asin='0007321198';
```

### Write a CQL solr_query to find the document where ranks.category = 'Movies' and ranks.rank = 1.

In [ ]:
CREATE TYPE amazon.category_rank (
    category text,
    rank int
);

ALTER TABLE amazon.metadata ADD ranks set<frozen<category_rank>>;

UPDATE amazon.metadata SET ranks = ranks + { { category:'Movies', rank:1 } } WHERE asin='0007321198';

### Answer:

Many ways to write this query, for example:

In [ ]:
select * from amazon.metadata where solr_query = '{"q":"*:*", "fq":"{!tuple}ranks.rank:1", "fq":"{!tuple}ranks.category:\"Movies\""}';

### What's the reason if you see the following error:
```
ServerError: <ErrorMessage code=0000 [Server error] message="undefined field ranks.rank">
```

### Answer:

Because CQL table schema was changed by "ALTER TABLE", a reindex is needed to get the solr core updated.

### Take a look at the new solr core schema for amazon.metadata, and see how UDT/tuple maps to Solr TupleField. Now rerun the previous query should succeed.

### Any other way of writing this solr query on UDT?

### Answer:

In [ ]:
select * from amazon.metadata where solr_query = '{!tuple}ranks.category:"Movies" AND ranks.rank:1';

### Why the following query doesn't work?

In [ ]:
select * from amazon.metadata where solr_query = '{!tuple}ranks.category:"Movies" AND {!tuple}ranks.rank:1';

### Answer:

According to [UDT Query Examples](http://docs.datastax.com/en/datastax_enterprise/4.8/datastax_enterprise/srch/srchTupleUDTqueries.html), You can query across UDTs or within a UDT:

-Across UDTs syntax: ({!tuple}cond1 AND {!tuple}condN...) *notice the parenthesis*

-Within a UDT: {!tuple}cond1 AND condN...

So the correct syntax should be either:
```
select * from amazon.metadata where solr_query = '{!tuple}ranks.category:"Movies" AND ranks.rank:1';
```
or
```
select * from amazon.metadata where solr_query = '({!tuple}ranks.category:"Movies" AND {!tuple}ranks.rank:1)';
```